In [153]:
# import libraries
import vtrigU as vtrig
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from math import ceil, log
from scipy import constants
from vtrigU_helper_functions import *
from tempfile import TemporaryFile

# define some constants
c = constants.c
antsLocations = ants_locations()

def convert2db(arr):
    return 20*np.log10((np.abs(arr/np.max(np.abs(arr)))))

In [149]:
# initialize the device
vtrig.Init()

# set setting structure
vtrigSettings = vtrig.RecordingSettings(
        vtrig.FrequencyRange(62.0*1000, # Start Frequency (in MHz)
                             66.5*1000, # Stop  Frequency (in MHz)
                             150),      # Number of Frequency Points (Maximum: 150)
        10.0,                           # RBW (in KHz)
        vtrig.VTRIG_U_TXMODE__HIGH_RATE  # Tx Mode (LOW: 20 Tx, MED: 10 Tx, HIGH: 4 Tx)
        ) 

# validate settings
vtrig.ValidateSettings(vtrigSettings)

# apply settings
vtrig.ApplySettings(vtrigSettings)

# get antenna pairs and convert to numpy matrix
TxRxPairs = np.array(vtrig.GetAntennaPairs(vtrigSettings.mode))

# get used frequencies in Hz
freq = np.array(vtrig.GetFreqVector_MHz()) * 1e6

# define constants
N_txrx = TxRxPairs.shape[0]
N_freq = freq.shape[0]

Nfft = 2**(ceil(log(freq.shape[0],2))+1)
Ts = 1/Nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
time_vec = np.linspace(0,Ts*(Nfft-1),num=Nfft)
dist_vec = time_vec*(c/2) # distance in meters

In [103]:
xgrid = np.array([0.18,0.18])
ygrid = np.arange(-0.3, 0.3+0.03, 0.03)
zgrid = np.arange( 0.1, 1+0.02,   0.02)
[Xgrid,Ygrid,Zgrid] = np.meshgrid(xgrid,ygrid,zgrid)

src = np.stack((Xgrid, Ygrid, Zgrid),3).reshape((-1,3,1,1))
src2 = np.transpose(src,[2,1,3,0])
 
# Use Radar equation to find total loss
# Pe/Ps = (Gtx*Grx*RCS*lambda^2)/((4pi)^3*|Rtx|^2*|Rrx|^2)
Rvec = src2 - np.expand_dims(antsLocations,axis=[2,3])

Rmag = np.expand_dims(norm(Rvec,axis=1),axis=1)
Rtheta = np.expand_dims(np.arctan2(norm(Rvec[:,0:1,:,:],axis=1),Rvec[:,2,:,:]),axis=1)
Rphi = np.expand_dims(np.arctan2(Rvec[:,1,:,:],Rvec[:,0,:,:]),axis=1)
Sphase = 2*np.pi*Rmag*np.expand_dims(freq,axis=[0,2,3])/c #Electrical Length in Radians
curRCS = 1 #m^2
curLambda = c/freq 
csf = np.sqrt(curRCS)*curLambda/((4*np.pi)**(3/2)) 
Smag = 10**(5.8/20)*RadiationPattern(Rtheta,Rphi)/Rmag

H2 = np.zeros((len(TxRxPairs),len(freq),1,src2.shape[3]))
for i in range(len(TxRxPairs)):
    tx = TxRxPairs[i,0]-1
    rx = TxRxPairs[i,1]-1
    H2[i,:,:,:] = 1/(csf.reshape(1,-1,1,1)*Smag[tx,:,:,:]*Smag[rx,:,:,:]*np.exp(-1j*(Sphase[tx,:,:,:]+Sphase[rx,:,:,:])))

H2 = np.transpose(H2,[3,0,1,2]).reshape((src2.shape[3],-1))

c:\Users\SCKY\.conda\envs\vtrigU\lib\site-packages\ipykernel_launcher.py:26: ComplexWarning: Casting complex values to real discards the imaginary part


In [152]:
nrecs = 10
calFrame = []
for i in range(nrecs):
    vtrig.Record()
    rec = vtrig.GetRecordingResult()
    recArr = []
    for key in rec.keys():
        recArr.append(rec[key])
    calFrame.append(np.array(recArr))
calFrame = np.mean(calFrame,axis=0)

nframes = 100
for i in range(nframes):
    vtrig.Record()
    rec = vtrig.GetRecordingResult()
    recArr = []
    for key in rec.keys():
        recArr.append(rec[key])
    X = np.array(recArr)


    x = np.fft.ifft(X,Nfft,1)
    PDP = norm(x,axis=0)
    plt.figure(1,figsize=(10,8))
    plt.plot(dist_vec,convert2db(PDP))
    plt.ylim((-50,50))
    plt.savefig("PDP.jpeg")
    plt.close()
